In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from drcomp.reducers import PCA, KernelPCA, AutoEncoder
from drcomp import estimate_intrinsic_dimension, DimensionalityReducer
from sklearn.datasets import fetch_lfw_people
import sklearn.manifold as manifold
import matplotlib.pyplot as plt
import time

In [ ]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, data_home="../data/raw")
image_shape = (62, 47)
X = lfw_people.data
X.shape

In [ ]:
intrinsic_dim = estimate_intrinsic_dimension(X)
intrinsic_dim

In [ ]:
tsne = manifold.TSNE(
    n_components=intrinsic_dim, perplexity=30, n_iter=1000, method="exact"
)
mds = manifold.MDS(n_components=intrinsic_dim, n_init=1, max_iter=100)
isomap = manifold.Isomap(n_components=intrinsic_dim, n_neighbors=10)

n_neighbors = 15
lle = manifold.LocallyLinearEmbedding(
    method="standard", n_neighbors=n_neighbors, n_components=intrinsic_dim
)

In [ ]:
models = [mds, isomap, lle]  # tsne,

In [ ]:
for model in models:
    print(model.__class__.__name__)
    start = time.time()
    model.fit(X)
    end = time.time()
    print("Fit Time: ", end - start)

In [ ]:
# Y_tsne = tsne.embedding_
Y_mds = mds.embedding_
Y_isomap = isomap.embedding_
Y_lle = lle.embedding_

In [ ]:
manifold.trustworthiness(X, Y_tsne, n_neighbors=n_neighbors)

In [ ]:
manifold.trustworthiness(X, Y_mds, n_neighbors=n_neighbors)

In [ ]:
manifold.trustworthiness(X, Y_isomap, n_neighbors=n_neighbors)

In [ ]:
manifold.trustworthiness(X, Y_lle, n_neighbors=n_neighbors)

In [ ]:
from pyDRMetrics.pyDRMetrics import DRMetrics

In [ ]:
drm_tsne = DRMetrics(X, Y_tsne)

In [ ]:
drm_tsne.report()

In [ ]:
drm_mds = DRMetrics(X, Y_mds)
drm_mds.report()

In [ ]:
drm_isomap = DRMetrics(X, Y_isomap)
drm_isomap.report()

In [ ]:
non_metrics_mds = manifold.MDS(n_components=intrinsic_dim, metric=False)

In [ ]:
Y = non_metrics_mds.fit_transform(X)
trustworthiness = manifold.trustworthiness(X, Y, n_neighbors=n_neighbors)